In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,r2_score
import time
import xgboost as xgb

data=pd.read_csv("Data/Flight_Data_Processed.csv")
X=data.drop(columns=['totalFare'],axis=1)
Y=data['totalFare']
del data

#scaling

'''standard=StandardScaler()

standard=standard.fit(X)
X=standard.transform(X)
'''
'''#train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)
#Scaling
standard=StandardScaler()

standard=standard.fit(X_train)
X_train=standard.transform(X_train)
X_test=standard.transform(X_test)
'''

'#train test split\nX_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)\n\n\nX_train=X_train.reset_index(drop=True)\nX_test=X_test.reset_index(drop=True)\ny_train=y_train.reset_index(drop=True)\ny_test=y_test.reset_index(drop=True)\n#Scaling\nstandard=StandardScaler()\n\nstandard=standard.fit(X_train)\nX_train=standard.transform(X_train)\nX_test=standard.transform(X_test)\n'

In [64]:
start_time=time.time()

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize lists to store MSE for each fold
mse_scores = []
r2_scores=[]

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    
    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate MSE and store it
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

    # Calculate r2 score and store it
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Calculate average MSE across all folds
avg_mse = np.mean(mse_scores)
print(mse_scores)
print("Average MSE:", avg_mse)

# Calculate average r2 across all folds
avg_r2 = np.mean(r2_scores)
print(r2_scores)
print("Average MSE:", avg_r2)
end_time=time.time()

print(f"time taken {end_time-start_time}")

[21887.60049952567, 21897.38157323084, 21791.798839875326, 22017.205386514273, 21672.711169419606]
Average MSE: 21853.339493713145
[0.5091286058474906, 0.5072995579477078, 0.5083645267278152, 0.5099122115473032, 0.5077131593987708]
Average MSE: 0.5084836122938176
time taken 4.054746389389038


In [65]:
start_time=time.time()

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize lists to store MSE for each fold
mse_scores = []
r2_scores=[]

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate MSE and store it
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

    # Calculate r2 score and store it
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Calculate average MSE across all folds
avg_mse = np.mean(mse_scores)
print(mse_scores)
print("Average MSE:", avg_mse)

# Calculate average r2 across all folds
avg_r2 = np.mean(r2_scores)
print(r2_scores)
print("Average MSE:", avg_r2)
end_time=time.time()

print(f"time taken {end_time-start_time}")

[17562.532523475704, 17611.141274142694, 17444.51336861179, 17802.922208774915, 17582.053905015]
Average MSE: 17600.632656004018
[0.6061265452631899, 0.6037417961687723, 0.6064417789004573, 0.6037192450119926, 0.6006307793926029]
Average MSE: 0.6041320289474029
time taken 138.4546570777893


In [75]:
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)
#Scaling
standard=StandardScaler()

standard=standard.fit(X_train)
X_train=standard.transform(X_train)
X_test=standard.transform(X_test)


param_grid = {
    'max_depth': [10,12,15,16],
    'min_samples_split': [2],
    'min_samples_leaf': [4,5]
}


tree_regressor = DecisionTreeRegressor(random_state=42)

# Create GridSearchCV instance
grid_search = GridSearchCV(tree_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best estimator
best_regressor = grid_search.best_estimator_

# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

# Evaluate the best model on the test set
y_pred = best_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test set MSE:", mse)

r2 = r2_score(y_test, y_pred)
print(r2)



Best parameters: {'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2}
Test set MSE: 15098.975600782398
0.6602045349376263


In [27]:
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


X_train=X_train.reset_index(drop=True)
X_test=X_test.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_test=y_test.reset_index(drop=True)
#Scaling
standard=StandardScaler()

standard=standard.fit(X_train)
X_train=standard.transform(X_train)
X_test=standard.transform(X_test)


xgb_regressor=xgb.XGBRegressor(n_estimators=50,max_depth=8,learning_rate=0.05,objective='reg:squarederror', device='cuda:0')
xgb_regressor.fit(X_train, y_train)



# Evaluate the best model on the test set
y_pred = xgb_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test set MSE:", mse)

r2 = r2_score(y_test, y_pred)
print(r2)



Test set MSE: 16548.342766237845
0.627587196976861


In [ ]:
RandomForestRegressor()